In [1]:
import pandas as pd
import os
import re
import json

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
from tqdm import tqdm
import torch.nn.functional as F
from gensim.models import Word2Vec
from sklearn import metrics
from sklearn.metrics import classification_report
import itertools
from sklearn.neighbors import NearestNeighbors

/home/eunchong/anaconda3/envs/network_x/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
tripod = pd.read_csv('TRIPOD_screenplays_test.csv')

In [4]:
tripod

,movie_name,tp1,tp2,tp3,tp4,tp5
0,The Back-up Plan,"[9, 10]","[40, 41]",[82],"[106, 107, 111]",[131]
1,The Shining (film),[6],[38],[76],"[177, 178]","[223, 225, 227, 228, 229]"
2,Juno (film),[3],"[31, 32]","[39, 40, 41]","[86, 87, 89]",[107]
3,Soldier (1998 American film),[35],[51],[109],[210],[223]
4,Panic Room,"[17, 18, 20]","[56, 58]",[135],[148],"[159, 160]"
5,Arbitrage (film),"[35, 36, 37]","[57, 62]","[67, 87, 88]",[105],[109]
6,The Breakfast Club,[5],[20],[31],[31],"[34, 39]"
7,Slumdog Millionaire,[32],[108],[139],"[150, 157]","[188, 191]"
8,Total Recall (1990 film),[16],"[51, 53, 55]",[72],[112],"[145, 146, 147, 148, 149, 150, 151, 152, 153, ..."
9,Unforgiven,"[6, 7]",[53],"[88, 91]",[95],[98]


In [5]:
tripod

,movie_name,tp1,tp2,tp3,tp4,tp5
0,The Back-up Plan,"[9, 10]","[40, 41]",[82],"[106, 107, 111]",[131]
1,The Shining (film),[6],[38],[76],"[177, 178]","[223, 225, 227, 228, 229]"
2,Juno (film),[3],"[31, 32]","[39, 40, 41]","[86, 87, 89]",[107]
3,Soldier (1998 American film),[35],[51],[109],[210],[223]
4,Panic Room,"[17, 18, 20]","[56, 58]",[135],[148],"[159, 160]"
5,Arbitrage (film),"[35, 36, 37]","[57, 62]","[67, 87, 88]",[105],[109]
6,The Breakfast Club,[5],[20],[31],[31],"[34, 39]"
7,Slumdog Millionaire,[32],[108],[139],"[150, 157]","[188, 191]"
8,Total Recall (1990 film),[16],"[51, 53, 55]",[72],[112],"[145, 146, 147, 148, 149, 150, 151, 152, 153, ..."
9,Unforgiven,"[6, 7]",[53],"[88, 91]",[95],[98]


In [6]:
path1 = 'word2vec_tripod_dim=384/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]
    print(len(mov_emb[movie_name][0][0]))

/tmp/ipykernel_2131820/3660590199.py:25: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484810403/work/torch/csrc/utils/tensor_numpy.cpp:172.)
  emb = torch.from_numpy(emb)


145
105
74
173
99
101
107
131
199
127
117
123
91
39
175


In [7]:
mov_scene_tp = pd.DataFrame(columns = ['Movie_name','stage1','stage2','stage3','stage4','stage5','stage6'])
mov_scene_tp

,Movie_name,stage1,stage2,stage3,stage4,stage5,stage6


In [8]:
mov_scene_tp = pd.DataFrame(columns = ['Movie_name','stage1','stage2','stage3','stage4','stage5','stage6'])
for movie_name in tripod['movie_name']:
    max_scene = mov_emb[movie_name][1]
    stage = {}
    stage[1] = []
    stage[2] = []
    stage[3] = []
    stage[4] = []
    stage[5] = []
    stage[6] = []
    last_tp = 0
    for i in range(1,6): 
        tp = [tp.replace('[','').replace(']','').split(', ') for tp in tripod[tripod['movie_name']==movie_name]['tp'+str(i)].values]
        tp = list(map(int, tp[0]))
        for j in range(last_tp, tp[-1]+1):
            if 'scene_' + str(j) in mov_emb[movie_name][2]:
                stage[i].append(j)
        last_tp = tp[0]
    for i in range(tp[0], max_scene+1):
        if 'scene_' + str(i) in mov_emb[movie_name][2]:
            stage[6].append(i)
    
    
    mov_dict = {'Movie_name':movie_name, 'stage1':[stage[1]], 'stage2':[stage[2]], 'stage3':[stage[3]], 'stage4':[stage[4]], 'stage5':[stage[5]], 'stage6':[stage[6]]}
    mov_scene_tp = pd.concat([mov_scene_tp, pd.DataFrame(mov_dict)])

    

In [9]:
mov_scene_tp

,Movie_name,stage1,stage2,stage3,stage4,stage5,stage6
0,The Back-up Plan,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21...","[40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 52, 5...","[82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 9...","[106, 107, 108, 109, 110, 111, 112, 113, 114, ...","[131, 132]"
0,The Shining (film),"[1, 3, 4, 6]","[6, 7, 8, 10, 12, 13, 15, 17, 18, 19, 20, 22, ...","[38, 39, 41, 42, 44, 45, 46, 47, 48, 49, 51, 5...","[76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 88, 8...","[178, 179, 182, 183, 184, 187, 188, 189, 191, ...","[223, 225, 228, 229, 230]"
0,Juno (film),"[1, 2, 3]","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, ...","[31, 32, 33, 34, 35, 37, 38, 40, 41]","[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 5...","[86, 87, 88, 89, 90, 92, 93, 94, 95, 96, 97, 9...","[107, 108, 109, 110, 111, 112, 113, 114, 115, ..."
0,Soldier (1998 American film),"[1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 16, 17, ...","[35, 36, 37, 40, 42, 44, 45, 46, 48, 49, 50, 51]","[51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 6...","[109, 111, 112, 113, 114, 115, 116, 117, 118, ...","[210, 211, 212, 213, 214, 215, 216, 218, 219, ...","[223, 224, 225, 226, 228, 229, 230]"
0,Panic Room,"[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...","[17, 18, 23, 25, 26, 27, 28, 30, 31, 33, 34, 3...","[56, 57, 58, 60, 61, 62, 63, 64, 65, 66, 68, 6...","[135, 136, 137, 138, 139, 140, 141, 142, 143, ...","[148, 149, 150, 151, 152, 153, 154, 155, 156, ...","[159, 160, 161, 162, 164, 166, 167]"
0,Arbitrage (film),"[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 1...","[35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 4...","[57, 58, 59, 60, 61, 62, 64, 65, 67, 68, 69, 7...","[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 7...","[105, 106, 107, 108]",[]
0,The Breakfast Club,"[1, 2, 3, 4, 5]","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18...","[20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31]",[31],"[31, 32, 33, 35, 36, 37, 38, 39]","[35, 36, 37, 38, 39, 40, 41, 42]"
0,Slumdog Millionaire,"[1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...","[32, 33, 34, 35, 36, 37, 38, 40, 41, 43, 44, 4...","[108, 109, 110, 111, 116, 117, 118, 119, 120, ...","[139, 140, 141, 142, 143, 144, 145, 146, 147, ...","[150, 151, 152, 153, 154, 155, 156, 157, 159, ...","[188, 189, 191, 192, 193, 194]"
0,Total Recall (1990 film),"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...","[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 2...","[51, 52, 53, 54, 55, 56, 58, 59, 61, 62, 63, 6...","[72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 8...","[112, 113, 114, 115, 116, 117, 118, 119, 120, ...","[145, 147, 151, 160]"
0,Unforgiven,"[1, 2, 3, 4, 5, 6, 7]","[6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 1...","[53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 6...","[88, 89, 90, 91, 92, 93, 94, 95]","[95, 96, 97, 98]","[98, 99, 100, 101, 102, 103, 104, 105, 106]"


In [10]:
num = {}
for i in range(1,7):
    n = 0
    for n_list in mov_scene_tp['stage'+str(i)].values:
        if not n_list:
            continue
        else:
            n += n_list[-1]-n_list[0]+1
  
    num[i] = n

In [11]:
num

{1: 293, 2: 462, 3: 562, 4: 542, 5: 284, 6: 90}

In [12]:
def what_stage(num, df):
    target = []
    for i in range(1,7):
        stage_list = df['stage'+str(i)].values[0]
        if num in stage_list:
            target.append(i)
    return set(target)

In [13]:
def indice2scene(num, node_id):
            
    return int(node_id[num][6:])

In [14]:
num_sample = 2

# Doc2vec

In [15]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [16]:
path1 = 'doc2vec/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Doc2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.dv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.dv.index_to_key:
            emb = w2v.dv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    node_ids = [i for i in w2v.dv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]
    print(len(mov_emb[movie_name][0][0]))


total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=num_sample, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    an = 0
    for indice in indices:
        an = 0
        indice_len += 1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                an = 1
        for i in range(1,7):
            if i in target:
                stage_ans[i-1] +=an
        if an:
            ans += 1
        for i in range(1,7):
            if i in target:
                stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')
    

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175
The Back-up Plan :   0.28346456692913385
The Shining (film) :   0.5838150289017341
Juno (film) :   0.4
Soldier (1998 American film) :   0.592964824120603
Panic Room :   0.46206896551724136
Arbitrage (film) :   0.5454545454545454
The Breakfast Club :   0.3333333333333333
Slumdog Millionaire :   0.37142857142857144
Total Recall (1990 film) :   0.4580152671755725
Unforgiven :   0.5544554455445545
Die Hard :   0.5514018691588785
Moon (film) :   0.3162393162393162
One Eight Seven :   0.3170731707317073
The Crying Game :   0.5604395604395604
The Last Temptation of Christ (film) :   0.40540540540540543
stage 1 :  0.39615384615384613
stage 2 :  0.42961165048543687
stage 3 :  0.5372549019607843
stage 4 :  0.5402061855670103
stage 5 :  0.3537117903930131
stage 6 :  0.15492957746478872
Total 0.4584717607973422 %


In [33]:
stage_num

[260, 412, 510, 485, 229, 71]

In [39]:
for i in stage_num:
    total = sum(stage_num)
    print(1-((total-i)/total)**5)

0.507795243822782
0.6912335054671074
0.7770148409248745
0.7572164716896834
0.4614487218218206
0.16791083786109484


In [40]:
1-(5/6)**5

0.598122427983539

In [34]:
sum(stage_num)

1967

# Sum

In [15]:
import pickle
with open('emb_sum.pickle','rb') as fw:
    movie_emb = pickle.load(fw)

In [17]:
path1 = 'word2vec_tripod_dim=384/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    print(scene_num)
    embs = torch.zeros(1,scene_num,384)
    k= 0

    for i in range(len(movie_emb[movie_name])):
        if np.isnan(movie_emb[movie_name][i][0]):
            continue
        embs[0][k] = torch.from_numpy(np.array(movie_emb[movie_name][i]))  
        k+=1
        if k ==scene_num:
            break

    
#    for i in range(1,len(movie_emb[movie_name])+1):
#        emb_name = base + str(i)
#        if emb_name in w2v.wv.index_to_key:
#            emb = np.nan_to_num(movie_emb[movie_name][k])
#            emb = torch.from_numpy(emb)
#            embs[0][k] = emb
#            k += 1
#    print(len(embs[0]))        
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]

total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=6, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    an = 0
    for indice in indices:
        an = 0
        indice_len += 1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                an = 1
        for i in range(1,7):
            if i in target:
                stage_ans[i-1] +=an
        if an:
            ans += 1
        for i in range(1,7):
            if i in target:
                stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')
    

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175
The Back-up Plan :   0.7716535433070866
The Shining (film) :   0.8497109826589595
Juno (film) :   0.8095238095238095
Soldier (1998 American film) :   0.7939698492462312
Panic Room :   0.8413793103448276
Arbitrage (film) :   0.8383838383838383
The Breakfast Club :   0.8461538461538461
Slumdog Millionaire :   0.8
Total Recall (1990 film) :   0.7938931297709924
Unforgiven :   0.7524752475247525
Die Hard :   0.9345794392523364
Moon (film) :   0.7521367521367521
One Eight Seven :   0.8292682926829268
The Crying Game :   0.8461538461538461
The Last Temptation of Christ (film) :   0.7972972972972973
stage 1 :  0.6519823788546255
stage 2 :  0.8539944903581267
stage 3 :  0.874326750448833
stage 4 :  0.897489539748954
stage 5 :  0.7865612648221344
stage 6 :  0.4520547945205479
Total 0.8150609080841639 %


# 평균

In [143]:
import pickle
with open('emb_avg.pickle','rb') as fw:
    movie_emb = pickle.load(fw)

In [28]:
import pickle
with open('emb_avg2.pickle','rb') as fw:
    movie_emb = pickle.load(fw)

In [34]:
path1 = 'word2vec_tripod_dim=384/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    print(scene_num)
    embs = torch.zeros(1,scene_num,384)
    k= 0

    for i in range(len(movie_emb[movie_name])):
        if np.isnan(movie_emb[movie_name][i][0]):
            continue
        embs[0][k] = torch.from_numpy(np.array(movie_emb[movie_name][i]))  
        k+=1
        if k ==scene_num:
            break

    
#    for i in range(1,len(movie_emb[movie_name])+1):
#        emb_name = base + str(i)
#        if emb_name in w2v.wv.index_to_key:
#            emb = np.nan_to_num(movie_emb[movie_name][k])
#            emb = torch.from_numpy(emb)
#            embs[0][k] = emb
#            k += 1
#    print(len(embs[0]))        
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]

total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=2, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    for indice in indices:
        indice_len += len(indice)-1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                ans +=1
                for i in range(1,7):
                    if i in target:
                        stage_ans[i-1] +=1
            for i in range(1,7):
                if i in target:
                    stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')

145
167


IndexError: index 145 is out of bounds for dimension 0 with size 145

In [32]:
path1 = 'word2vec_tripod_dim=384/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    print(scene_num)
    embs = torch.zeros(1,scene_num,384)
    k = 0

    for i in range(1,len(movie_emb[movie_name])+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = np.nan_to_num(movie_emb[movie_name][k])
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
#    print(len(embs[0]))        
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]

total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=4, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    an = 0
    for indice in indices:
        an = 0
        indice_len += 1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                an = 1
        for i in range(1,7):
            if i in target:
                stage_ans[i-1] +=an
        if an:
            ans += 1
        for i in range(1,7):
            if i in target:
                stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')
    

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175
The Back-up Plan :   0.6929133858267716
The Shining (film) :   0.7861271676300579
Juno (film) :   0.6761904761904762
Soldier (1998 American film) :   0.7839195979899497
Panic Room :   0.8068965517241379
Arbitrage (film) :   0.8080808080808081
The Breakfast Club :   0.8461538461538461
Slumdog Millionaire :   0.6914285714285714
Total Recall (1990 film) :   0.7175572519083969
Unforgiven :   0.7920792079207921
Die Hard :   0.8317757009345794
Moon (film) :   0.6495726495726496
One Eight Seven :   0.7886178861788617
The Crying Game :   0.8021978021978022
The Last Temptation of Christ (film) :   0.7297297297297297
stage 1 :  0.7192307692307692
stage 2 :  0.7888349514563107
stage 3 :  0.8235294117647058
stage 4 :  0.7814432989690722
stage 5 :  0.6899563318777293
stage 6 :  0.4647887323943662
Total 0.7558139534883721 %


# Edge

In [161]:
path1 = 'word2vec_tripod_edge/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,400)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]
    print(len(mov_emb[movie_name][0][0]))

total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=num_sample, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    for indice in indices:
        indice_len += len(indice)-1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                ans +=1
                for i in range(1,7):
                    if i in target:
                        stage_ans[i-1] +=1
            for i in range(1,7):
                if i in target:
                    stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175
The Back-up Plan :   0.33070866141732286
The Shining (film) :   0.5664739884393064
Juno (film) :   0.2761904761904762
Soldier (1998 American film) :   0.5326633165829145
Panic Room :   0.503448275862069
Arbitrage (film) :   0.494949494949495
The Breakfast Club :   0.3333333333333333
Slumdog Millionaire :   0.3028571428571429
Total Recall (1990 film) :   0.4580152671755725
Unforgiven :   0.46534653465346537
Die Hard :   0.5981308411214953
Moon (film) :   0.23931623931623933
One Eight Seven :   0.45528455284552843
The Crying Game :   0.4835164835164835
The Last Temptation of Christ (film) :   0.527027027027027
stage 1 :  0.4
stage 2 :  0.4320388349514563
stage 3 :  0.5509803921568628
stage 4 :  0.5051546391752577
stage 5 :  0.31004366812227074
stage 6 :  0.15492957746478872
Total 0.4435215946843854 %


In [171]:
path1 = 'word2vec_tripod_edge/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,400)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]
    print(len(mov_emb[movie_name][0][0]))

total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=num_sample, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    an = 0
    for indice in indices:
        an = 0
        indice_len += 1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                an = 1
        for i in range(1,7):
            if i in target:
                stage_ans[i-1] +=an
        if an:
            ans += 1
        for i in range(1,7):
            if i in target:
                stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175
The Back-up Plan :   0.6141732283464567
The Shining (film) :   0.7976878612716763
Juno (film) :   0.7523809523809524
Soldier (1998 American film) :   0.7386934673366834
Panic Room :   0.7448275862068966
Arbitrage (film) :   0.8080808080808081
The Breakfast Club :   0.717948717948718
Slumdog Millionaire :   0.6628571428571428
Total Recall (1990 film) :   0.7480916030534351
Unforgiven :   0.7227722772277227
Die Hard :   0.8037383177570093
Moon (film) :   0.5982905982905983
One Eight Seven :   0.6991869918699187
The Crying Game :   0.7692307692307693
The Last Temptation of Christ (film) :   0.7702702702702703
stage 1 :  0.7153846153846154
stage 2 :  0.75
stage 3 :  0.7666666666666667
stage 4 :  0.7649484536082474
stage 5 :  0.6899563318777293
stage 6 :  0.4788732394366197
Total 0.7275747508305648 %


# No_embedding

In [619]:
path1 = 'word2vec_tripod_noembedding/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]
    print(len(mov_emb[movie_name][0][0]))


total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=num_sample, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    for indice in indices:
        indice_len += len(indice)-1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                ans +=1
                for i in range(1,7):
                    if i in target:
                        stage_ans[i-1] +=1
            for i in range(1,7):
                if i in target:
                    stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')
    

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175
The Back-up Plan :   0.6299212598425197
The Shining (film) :   0.5028901734104047
Juno (film) :   0.5047619047619047
Soldier (1998 American film) :   0.6633165829145728
Panic Room :   0.5241379310344828
Arbitrage (film) :   0.6060606060606061
The Breakfast Club :   0.3076923076923077
Slumdog Millionaire :   0.5657142857142857
Total Recall (1990 film) :   0.648854961832061
Unforgiven :   0.6633663366336634
Die Hard :   0.616822429906542
Moon (film) :   0.42735042735042733
One Eight Seven :   0.6504065040650406
The Crying Game :   0.7802197802197802
The Last Temptation of Christ (film) :   0.5675675675675675
stage 1 :  0.5269230769230769
stage 2 :  0.6116504854368932
stage 3 :  0.6470588235294118
stage 4 :  0.6515463917525773
stage 5 :  0.48034934497816595
stage 6 :  0.4225352112676056
Total 0.5869324473975637 %


In [577]:
path1 = 'word2vec_tripod_noembedding/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]
    print(len(mov_emb[movie_name][0][0]))

total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=num_sample, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    an = 0
    for indice in indices:
        an = 0
        indice_len += 1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                an = 1
        for i in range(1,7):
            if i in target:
                stage_ans[i-1] +=an
        if an:
            ans += 1
        for i in range(1,7):
            if i in target:
                stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175
The Back-up Plan :   0.9133858267716536
The Shining (film) :   0.861271676300578
Juno (film) :   0.9047619047619048
Soldier (1998 American film) :   0.8944723618090452
Panic Room :   0.9172413793103448
Arbitrage (film) :   0.9393939393939394
The Breakfast Club :   0.8717948717948718
Slumdog Millionaire :   0.8228571428571428
Total Recall (1990 film) :   0.816793893129771
Unforgiven :   0.8811881188118812
Die Hard :   0.9345794392523364
Moon (film) :   0.9145299145299145
One Eight Seven :   0.967479674796748
The Crying Game :   0.9120879120879121
The Last Temptation of Christ (film) :   0.918918918918919
stage 1 :  0.9576923076923077
stage 2 :  0.9466019417475728
stage 3 :  0.8960784313725491
stage 4 :  0.9257731958762887
stage 5 :  0.8122270742358079
stage 6 :  0.5070422535211268
Total 0.8942414174972314 %


In [581]:
path1 = 'word2vec_tripod_noembedding/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]
    print(len(mov_emb[movie_name][0][0]))

total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=num_sample, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    an = 0
    for indice in indices:
        an = 0
        indice_len += 1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                an = 1
        for i in range(1,7):
            if i in target:
                stage_ans[i-1] +=an
        if an:
            ans += 1
        for i in range(1,7):
            if i in target:
                stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175
The Back-up Plan :   0.4881889763779528
The Shining (film) :   0.47398843930635837
Juno (film) :   0.6095238095238096
Soldier (1998 American film) :   0.5577889447236181
Panic Room :   0.593103448275862
Arbitrage (film) :   0.7373737373737373
The Breakfast Club :   0.5641025641025641
Slumdog Millionaire :   0.38285714285714284
Total Recall (1990 film) :   0.37404580152671757
Unforgiven :   0.5742574257425742
Die Hard :   0.7383177570093458
Moon (film) :   0.7094017094017094
One Eight Seven :   0.5609756097560976
The Crying Game :   0.5494505494505495
The Last Temptation of Christ (film) :   0.4189189189189189
stage 1 :  0.7038461538461539
stage 2 :  0.6577669902912622
stage 3 :  0.5333333333333333
stage 4 :  0.5587628865979382
stage 5 :  0.38427947598253276
stage 6 :  0.23943661971830985
Total 0.5459579180509413 %


In [584]:
path1 = 'word2vec_tripod_noembedding/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]
    print(len(mov_emb[movie_name][0][0]))

total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=num_sample, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    an = 0
    for indice in indices:
        an = 0
        indice_len += 1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                an = 1
        for i in range(1,7):
            if i in target:
                stage_ans[i-1] +=an
        if an:
            ans += 1
        for i in range(1,7):
            if i in target:
                stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175
The Back-up Plan :   0.7952755905511811
The Shining (film) :   0.7630057803468208
Juno (film) :   0.8476190476190476
Soldier (1998 American film) :   0.8190954773869347
Panic Room :   0.8344827586206897
Arbitrage (film) :   0.898989898989899
The Breakfast Club :   0.7948717948717948
Slumdog Millionaire :   0.6914285714285714
Total Recall (1990 film) :   0.6335877862595419
Unforgiven :   0.8316831683168316
Die Hard :   0.8504672897196262
Moon (film) :   0.8888888888888888
One Eight Seven :   0.8780487804878049
The Crying Game :   0.8461538461538461
The Last Temptation of Christ (film) :   0.8108108108108109
stage 1 :  0.8846153846153846
stage 2 :  0.8689320388349514
stage 3 :  0.8196078431372549
stage 4 :  0.8329896907216495
stage 5 :  0.6681222707423581
stage 6 :  0.43661971830985913
Total 0.805094130675526 %


# Text Rank

In [27]:
path1 = 'word2vec_tripod_rank3/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]
    print(len(mov_emb[movie_name][0][0]))

total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=6, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    an = 0
    for indice in indices:
        an = 0
        indice_len += 1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                an = 1
        for i in range(1,7):
            if i in target:
                stage_ans[i-1] +=an
        if an:
            ans += 1
        for i in range(1,7):
            if i in target:
                stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')
    

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175
The Back-up Plan :   0.8267716535433071
The Shining (film) :   0.8670520231213873
Juno (film) :   0.8380952380952381
Soldier (1998 American film) :   0.8693467336683417
Panic Room :   0.8137931034482758
Arbitrage (film) :   0.8787878787878788
The Breakfast Club :   0.7948717948717948
Slumdog Millionaire :   0.7542857142857143
Total Recall (1990 film) :   0.8931297709923665
Unforgiven :   0.9207920792079208
Die Hard :   0.9065420560747663
Moon (film) :   0.7008547008547008
One Eight Seven :   0.8943089430894309
The Crying Game :   0.9340659340659341
The Last Temptation of Christ (film) :   0.8648648648648649
stage 1 :  0.8423076923076923
stage 2 :  0.8786407766990292
stage 3 :  0.8705882352941177
stage 4 :  0.8948453608247423
stage 5 :  0.777292576419214
stage 6 :  0.5633802816901409
Total 0.8482834994462901 %


# My model

In [620]:
path1 = 'word2vec_tripod_dim=384/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]
    print(len(mov_emb[movie_name][0][0]))

total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=num_sample, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    for indice in indices:
        indice_len += len(indice)-1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                ans +=1
                for i in range(1,7):
                    if i in target:
                        stage_ans[i-1] +=1
            for i in range(1,7):
                if i in target:
                    stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')
    

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175
The Back-up Plan :   0.4881889763779528
The Shining (film) :   0.4797687861271676
Juno (film) :   0.580952380952381
Soldier (1998 American film) :   0.6633165829145728
Panic Room :   0.6137931034482759
Arbitrage (film) :   0.494949494949495
The Breakfast Club :   0.358974358974359
Slumdog Millionaire :   0.44
Total Recall (1990 film) :   0.5572519083969466
Unforgiven :   0.5148514851485149
Die Hard :   0.6355140186915887
Moon (film) :   0.38461538461538464
One Eight Seven :   0.6504065040650406
The Crying Game :   0.6813186813186813
The Last Temptation of Christ (film) :   0.43243243243243246
stage 1 :  0.4230769230769231
stage 2 :  0.5388349514563107
stage 3 :  0.5647058823529412
stage 4 :  0.6309278350515464
stage 5 :  0.537117903930131
stage 6 :  0.4647887323943662
Total 0.5420819490586932 %


In [ ]:
path1 = 'word2vec_tripod_dim=384/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]
    print(len(mov_emb[movie_name][0][0]))
total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=num_sample, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    an = 0
    for indice in indices:
        an = 0
        indice_len += 1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                an = 1
        for i in range(1,7):
            if i in target:
                stage_ans[i-1] +=an
        if an:
            ans += 1
        for i in range(1,7):
            if i in target:
                stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')

In [582]:
path1 = 'word2vec_tripod_dim=384/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]
    print(len(mov_emb[movie_name][0][0]))
total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=num_sample, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    an = 0
    for indice in indices:
        an = 0
        indice_len += 1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                an = 1
        for i in range(1,7):
            if i in target:
                stage_ans[i-1] +=an
        if an:
            ans += 1
        for i in range(1,7):
            if i in target:
                stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175
The Back-up Plan :   0.4094488188976378
The Shining (film) :   0.49710982658959535
Juno (film) :   0.5142857142857142
Soldier (1998 American film) :   0.5728643216080402
Panic Room :   0.5586206896551724
Arbitrage (film) :   0.6565656565656566
The Breakfast Club :   0.5897435897435898
Slumdog Millionaire :   0.37714285714285717
Total Recall (1990 film) :   0.44274809160305345
Unforgiven :   0.5643564356435643
Die Hard :   0.5794392523364486
Moon (film) :   0.48717948717948717
One Eight Seven :   0.5609756097560976
The Crying Game :   0.6813186813186813
The Last Temptation of Christ (film) :   0.47297297297297297
stage 1 :  0.7307692307692307
stage 2 :  0.5048543689320388
stage 3 :  0.4980392156862745
stage 4 :  0.5979381443298969
stage 5 :  0.388646288209607
stage 6 :  0.28169014084507044
Total 0.5210409745293466 %


In [585]:
path1 = 'word2vec_tripod_dim=384/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]
    print(len(mov_emb[movie_name][0][0]))
total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=num_sample, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    an = 0
    for indice in indices:
        an = 0
        indice_len += 1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                an = 1
        for i in range(1,7):
            if i in target:
                stage_ans[i-1] +=an
        if an:
            ans += 1
        for i in range(1,7):
            if i in target:
                stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175
The Back-up Plan :   0.8267716535433071
The Shining (film) :   0.7861271676300579
Juno (film) :   0.8095238095238095
Soldier (1998 American film) :   0.8090452261306532
Panic Room :   0.8551724137931035
Arbitrage (film) :   0.8888888888888888
The Breakfast Club :   0.8461538461538461
Slumdog Millionaire :   0.7028571428571428
Total Recall (1990 film) :   0.7709923664122137
Unforgiven :   0.8613861386138614
Die Hard :   0.822429906542056
Moon (film) :   0.7777777777777778
One Eight Seven :   0.8292682926829268
The Crying Game :   0.8681318681318682
The Last Temptation of Christ (film) :   0.7567567567567568
stage 1 :  0.9307692307692308
stage 2 :  0.8106796116504854
stage 3 :  0.792156862745098
stage 4 :  0.845360824742268
stage 5 :  0.759825327510917
stage 6 :  0.5492957746478874
Total 0.8078626799557033 %


In [407]:
import pickle
with open('emb_avg.pickle','rb') as fw:
    movie_emb = pickle.load(fw)

In [408]:
path1 = 'word2vec_tripod_dim=384/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    print(scene_num)
    embs = torch.zeros(1,scene_num,384)
    k = 0

    for i in range(1,len(movie_emb[movie_name])+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = np.nan_to_num(movie_emb[movie_name][k])
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
#    print(len(embs[0]))        
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175


In [400]:
len(mov_emb['Total Recall (1990 film)'][0][0])

131

In [208]:
num_sample = 4

In [210]:
path1 = 'word2vec_tripod_dim=384/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]
    print(len(mov_emb[movie_name][0][0]))

total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=num_sample, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    an = 0
    for indice in indices:
        an = 0
        indice_len += 1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                an = 1
        for i in range(1,7):
            if i in target:
                stage_ans[i-1] +=an
        if an:
            ans += 1
        for i in range(1,7):
            if i in target:
                stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175
The Back-up Plan :   0.6377952755905512
The Shining (film) :   0.7514450867052023
Juno (film) :   0.6857142857142857
Soldier (1998 American film) :   0.7085427135678392
Panic Room :   0.7931034482758621
Arbitrage (film) :   0.7676767676767676
The Breakfast Club :   0.6666666666666666
Slumdog Millionaire :   0.6171428571428571
Total Recall (1990 film) :   0.7099236641221374
Unforgiven :   0.801980198019802
Die Hard :   0.8037383177570093
Moon (film) :   0.7008547008547008
One Eight Seven :   0.7886178861788617
The Crying Game :   0.8241758241758241
The Last Temptation of Christ (film) :   0.7297297297297297
stage 1 :  0.7153846153846154
stage 2 :  0.720873786407767
stage 3 :  0.7843137254901961
stage 4 :  0.7793814432989691
stage 5 :  0.6506550218340611
stage 6 :  0.4788732394366197
Total 0.729235880398671 %


In [209]:
path1 = 'word2vec_tripod_extra+emb/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]
    print(len(mov_emb[movie_name][0][0]))

total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=num_sample, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    an = 0
    for indice in indices:
        an = 0
        indice_len += 1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                an = 1
        for i in range(1,7):
            if i in target:
                stage_ans[i-1] +=an
        if an:
            ans += 1
        for i in range(1,7):
            if i in target:
                stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175
The Back-up Plan :   0.6377952755905512
The Shining (film) :   0.7514450867052023
Juno (film) :   0.6952380952380952
Soldier (1998 American film) :   0.7839195979899497
Panic Room :   0.7448275862068966
Arbitrage (film) :   0.8484848484848485
The Breakfast Club :   0.6923076923076923
Slumdog Millionaire :   0.6342857142857142
Total Recall (1990 film) :   0.7557251908396947
Unforgiven :   0.8316831683168316
Die Hard :   0.7850467289719626
Moon (film) :   0.6324786324786325
One Eight Seven :   0.7886178861788617
The Crying Game :   0.7582417582417582
The Last Temptation of Christ (film) :   0.6891891891891891
stage 1 :  0.7076923076923077
stage 2 :  0.7451456310679612
stage 3 :  0.788235294117647
stage 4 :  0.797938144329897
stage 5 :  0.6419213973799127
stage 6 :  0.4647887323943662
Total 0.7353266888150609 %


In [30]:
path1 = 'word2vec_tripod_pos+extra/'
path2 = '../data/scriptbase/'
filenames = os.listdir(path1)
mov_emb = {}
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    if movie_name not in tripod['movie_name'].values:
        continue
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,400)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    node_ids = [i for i in w2v.wv.index_to_key if 'scene' in i]
    node_ids.sort()
    node_ids.sort(key=len)
    
    mov_emb[movie_name] = [embs,max_scene,node_ids]
    print(len(mov_emb[movie_name][0][0]))

total_num = 0
total_ans = 0
stage_num = [0]*6

stage_ans = [0]*6

for movie_name in tripod['movie_name'].values:
    name = movie_name
    X = mov_emb[name][0][0]
    nbrs = NearestNeighbors(n_neighbors=num_sample, algorithm='auto', metric ='mahalanobis',metric_params={'VI': np.cov(X.T)}).fit(X)
    _,indices = nbrs.kneighbors(X)
    node_id = mov_emb[name][2]
    indice_len = 0
    ans = 0
    an = 0
    for indice in indices:
        an = 0
        indice_len += 1
        target = what_stage(indice2scene(indice[0],node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
        for ind in indice[1:]:
            ind_ans = what_stage(indice2scene(ind,node_id),mov_scene_tp[mov_scene_tp['Movie_name']==name])
            if set(target) & set(ind_ans):
                an = 1
        for i in range(1,7):
            if i in target:
                stage_ans[i-1] +=an
        if an:
            ans += 1
        for i in range(1,7):
            if i in target:
                stage_num[i-1] += 1
                
    total_num += indice_len
    total_ans += ans
    print(movie_name, ':  ',ans/indice_len)
for i in range(6):
    print("stage",str(i+1),": ",stage_ans[i]/stage_num[i])
print('Total', total_ans/total_num,'%')

145
105
74
173
99
101
107
131
199
127
117
123
91
39
175
The Back-up Plan :   0.8188976377952756
The Shining (film) :   0.8728323699421965
Juno (film) :   0.819047619047619
Soldier (1998 American film) :   0.8743718592964824
Panic Room :   0.903448275862069
Arbitrage (film) :   0.9393939393939394
The Breakfast Club :   0.8974358974358975
Slumdog Millionaire :   0.8171428571428572
Total Recall (1990 film) :   0.8625954198473282
Unforgiven :   0.9108910891089109
Die Hard :   0.8785046728971962
Moon (film) :   0.7350427350427351
One Eight Seven :   0.8780487804878049
The Crying Game :   0.945054945054945
The Last Temptation of Christ (film) :   0.8648648648648649
stage 1 :  0.8615384615384616
stage 2 :  0.8713592233009708
stage 3 :  0.8960784313725491
stage 4 :  0.9010309278350516
stage 5 :  0.8209606986899564
stage 6 :  0.647887323943662
Total 0.8637873754152824 %
